In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from zoopt import Dimension, Objective, Parameter, Opt
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import seaborn as sns
from torchsummary import summary  # Import the summary function

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [26]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [27]:
dataset = ImageFolder(root='/home/idrone2/Desktop/alzheimer', transform=transform)
dataset

Dataset ImageFolder
    Number of datapoints: 5069
    Root location: /home/idrone2/Desktop/alzheimer
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [28]:
# Split dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [29]:
# Data loaders
train_loaders = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=1)
test_loaders = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=1)
val_loaders = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=1)

In [30]:
class CNN(nn.Module):
    def __init__(self, num_layers, dropout_rate):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 3
        out_channels = 32
        kernel_size = 3
        padding = 1
        
        for _ in range(num_layers):
            self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2, 2))
            in_channels = out_channels
            out_channels *= 2
        
        # Initialize flatten_size dynamically
        self.flatten_size = self._calculate_flatten_size()
        self.fc1 = nn.Linear(self.flatten_size, 256)
        self.fc2 = nn.Linear(256, len(dataset.classes))  # Ensure dataset.classes is defined
        self.dropout = nn.Dropout(p=dropout_rate)
    
    def _calculate_flatten_size(self):
        # Dummy input to calculate flatten_size
        dummy_input = torch.zeros(1, 3, 32, 32).to(device)
        x = dummy_input
        for layer in self.layers:
            x = layer(x)
        return int(torch.prod(torch.tensor(x.size())))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(-1, self.flatten_size)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [33]:
# Training function
def train_model(num_layers, dropout_rate, learning_rate):
    model = CNN(num_layers, dropout_rate).to(device)
    
    # Print the model summary
    print("Model Summary:")
    summary(model, input_size=(3, 32, 32))
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    num_epochs = 20

    train_losses = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loaders:
            # Move images and labels to the correct device
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)  # Forward pass

            # Ensure the shapes are correct
            if outputs.size(0) != labels.size(0):
                raise ValueError(f'Batch size mismatch: outputs.size(0)={outputs.size(0)}, labels.size(0)={labels.size(0)}')

            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_losses.append(running_loss / len(train_loaders))

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loaders:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

                # Ensure the shapes are correct
                if outputs.size(0) != labels.size(0):
                    raise ValueError(f'Batch size mismatch: outputs.size(0)={outputs.size(0)}, labels.size(0)={labels.size(0)}')

                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_losses.append(val_loss / len(val_loaders))
        val_accuracies.append(correct / total)

    return train_losses, val_losses, val_accuracies, model

In [35]:
# ZOOpt objective function
def objective_function(solution):
    # Convert ZOOpt Solution object to list of parameters
    params = solution.get_x()
    learning_rate = params[0]
    num_layers = int(params[1])
    dropout_rate = params[2]

    train_losses, val_losses, val_accuracies, _ = train_model(num_layers, dropout_rate, learning_rate)
    return -val_accuracies[-1]  # Return negative accuracy for minimization

# ZOOpt configuration
dim = Dimension(
    3,
    [[1e-5, 1e-2],  # learning_rate
     [1, 5],        # num_layers
     [0.1, 0.5]],   # dropout_rate
    [True, True, True]
)
objective = Objective(objective_function, dim)
budget = 20  # Number of function evaluations
parameter = Parameter(budget=budget)

# Run ZOOpt
solution = Opt.min(objective, parameter)
best_params = solution.get_x()  # Extract the best parameters
best_learning_rate = best_params[0]
best_num_layers = int(best_params[1])
best_dropout_rate = best_params[2]

print("Best parameters found by ZOOpt:")
print("Learning rate:", best_learning_rate)
print("Number of layers:", best_num_layers)
print("Dropout rate:", best_dropout_rate)


 #Final training with the best parameters found by ZOOpt
train_losses, val_losses, val_accuracies, final_model = train_model(best_num_layers, best_dropout_rate, best_learning_rate)


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
# Plotting training and validation losses
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.show()

# Plotting validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy over Epochs')
plt.show()

In [ ]:
# Confusion Matrix and Classification Report
y_true = []
y_pred = []
final_model.eval()
with torch.no_grad():
    for images, labels in test_loaders:
        images, labels = images.to(device), labels.to(device)
        outputs = final_model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=dataset.classes, yticklabels=dataset.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=dataset.classes))


In [ ]:
# ROC and AUC (One-vs-Rest)
y_true_bin = label_binarize(y_true, classes=range(len(dataset.classes)))
y_pred_bin = np.array([F.softmax(final_model(torch.tensor(img).unsqueeze(0).to(device)).cpu().detach(), dim=1).numpy() for img in dataset.imgs])

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(dataset.classes)

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
plt.figure(figsize=(10, 8))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(dataset.classes[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend